## References


http://schwitzsplinters.blogspot.com/2019/08/the-295-most-cited-contemporary-authors.html

http://philosophydata.com/about_me.html


In [1]:
import json
import uuid
import re
import pickle
import pandas as pd
import copy

import plotly.express as px

sys.path.append('src/')
import preprocessing
import get_ent_info
import exploratory_analysis
import get_reference


%load_ext autoreload
%autoreload 2


Decisions here: only keep the first date, ignore the reprint, translation, etc.

- Husik, Isaac, 1916–1918,“Studies in Gersonides,”
<em>Jewish Quarterly Review</em>, 7 (new series): 553–594; 8
(new series): 113–156, 231–268.
- Van Riel, Gerd, Carlos Steel, and James J. McEvoy (eds.), 1996,
<em>Iohannes Scottus Eriugena. The Bible and Hermeneutics. Proceedings
of the Ninth International Colloquium of the Society for the Promotion
of Eriugenian Studies Held at Leuven and Louvain-La-Neuve, June
7–10, 1995</em>. Leuven: Leuven University Press.
- Hume, David, 1739–40/2007, <i>A Treatise of Human Nature: A
Critical Edition</i>, David Fate Norton and Mary J. Norton (eds.),
Oxford Clarendon Press.
- Hutcheson, Francis, 1742/2002, <em>An Essay on the Nature and
Conduct of the Passions and Affections, with Illustrations on the
Moral Sense</em>, Aaron Garrett (ed.), Indianapolis: Liberty
- [LW] <em>Luther’s Works</em>, American edition, 55 vols. St
Louis and Philadephia: Concordia and Fortress Press,
1958–86; new series, vols 56–75, 2009–.
- Brecht, Martin, 1981 [1985], <em>Martin Luther: sein Weg zur
Reformation, 1483–1521</em>, Stuttgart: Calwer. Translated as
<em>Martin Luther: His Road to Reformation, 1483–1521</em>,
James L. Schaaf (trans.), Minneapolis, MN: Fortress Press, 1985.


**Patterns** 
- Corti, Guglielmo (ed.) (1966). <em>Il “Tractatus de
gratia” di Guglielmo d’Auvergne</em>. Rome: Lateran
University, 1966. An edition of <em>De gratia et libero
arbitrio</em>.
- 


## Design database

save extracted data as dictionaries

## Bibliography Data

get ref_list
Start with time (year)

In [2]:
# clean categories in bibliographies
with open('get_data/sep_entry_notext_list.json', 'r') as f:
    ent_list = json.load(f)

update_ent_list = []
for item in ent_list:
    url = item['url'].split('/')[-1]
    # we also want to differentiate each section in the bibliography
    cat_ref_list = list(item["bibliography"]["cat_ref_text"].keys())
    for i in range(len(cat_ref_list)):
        cat = cat_ref_list[i]
        update_cat = cat
        # e.g., <a id="Ref">References</a> / <a name="CriEdi">Critical Edition</a>
        if re.findall('\">(.*?)</a>', cat, re.DOTALL): # 814 items
            update_cat = re.findall('\">(.*?)</a>', cat, re.DOTALL)[0]

        item["bibliography"]["cat_ref_text"][update_cat] = item["bibliography"]["cat_ref_text"].pop(cat)
    update_cat_list = []
    for i in range(len(item["bibliography"]["categories"])):
        cat = item["bibliography"]["categories"][i]
        update_cat = item["bibliography"]["categories"][i]
        if re.findall('\">(.*?)</a>', cat, re.DOTALL): # 814 items
            update_cat = re.findall('\">(.*?)</a>', cat, re.DOTALL)[0]
        update_cat_list.append(update_cat)
    item["bibliography"]["categories"] = update_cat_list

    update_ent_list.append(item)
    # print(item["bibliography"]["cat_ref_text"].keys())

# save in json
with open('get_data/sep_entry_notext_list.json', 'w') as f:
    json.dump(update_ent_list, f, indent=4)

### Add missing authors

-----

In [3]:

with open('get_data/sep_entry_notext_list.json', 'r') as f:
    ent_list = json.load(f)
print(len(ent_list))

# biuld a dictionary
ent_ref_dict = {}

# BUG: "</h3>\n<ul class=, 2019, <em>Japanes

# open miss_author_cat_list
with open('get_data/to_check/miss_author_cat_dict.json', 'r') as f:
    miss_author_cat_dict = json.load(f)


# also correct errors: fill up the "---"
ref_list = []
author_pattern_1_list = [] # has the same author as the previous one

for item in ent_list:
    url = item['url'].split('/')[-1]
    ent_ref_dict[url] = {}
    # we also want to differentiate each section in the bibliography
    cat_ref_list = list(item["bibliography"]["cat_ref_text"].keys())
    for cat in cat_ref_list:
        listed_ref = item["bibliography"]["cat_ref_text"][cat]
        ent_ref_dict[url][cat] = {
            'notes': [],
            'ref_count': 0,
            'ref_list': []
        }

        if listed_ref and listed_ref[0] == "</h3>\n<ul class=\"hanging\">":
            listed_ref[0] = ""
        if listed_ref and re.findall('</h3>\n<p>(.*?)</p>\n<ul class=\"hanging\">', listed_ref[0], re.DOTALL): # there are nested sections
            
            ent_ref_dict[url][cat]["notes"].append("NOTES: " + re.findall('</h3>\n<p>(.*?)</p>\n<ul class=\"hanging\">', listed_ref[0], re.DOTALL)[0])
            listed_ref[0] = ""  

        
        # RECOVER: 

        # fix missing information of author

        for i in range(len(listed_ref)):
            s = listed_ref[i].strip() # in case " <em>"
            target = ""
            if len(s) < 10: # e.g., "DP"
                continue
                  
            if url+cat in miss_author_cat_dict:
                thinker_name = miss_author_cat_dict[url+cat]
                # can be an entry on thoughts
                #  indentation??? "\u201cThe Analysis of Sensations\u
                
                if s.startswith("<em>") or s.startswith("\u201c") or s[0].isdigit():
                    s = thinker_name + ",, " + s # ,, indicates that it is what we add
                elif "–––" in s or "\u2013\u2013\u2013" in s or "_______" in s: # need to be changed here
                    s = preprocessing.add_pre_author_name(listed_ref, i)
            else:
                if "–––" in s or "\u2013\u2013\u2013" in s or "_______" in s: # need to be changed here
                    s = preprocessing.add_pre_author_name(listed_ref, i)
                
            ref_list.append(s)
            # add the reference list (of each entry) to the dictionary
            ent_ref_dict[url][cat]['ref_list'].append(s)
        # add count 
        ent_ref_dict[url][cat]['ref_count'] = len(ent_ref_dict[url][cat]['ref_list'])

print(len(author_pattern_1_list))
print("ref_list:",len(ref_list), 
      "unique ref list:", len(list(set(ref_list))),
      "\nent_ref_dict:",len(ent_ref_dict))

# save dict
with open('get_data/ent_ref_dict.json', 'w') as f:
    json.dump(ent_ref_dict, f, indent=4)

# save ref list
with open('get_data/ref_list.json', 'w') as f:
    json.dump(ref_list, f, indent=4)


1796
target is empty
ERROR author is empty:
 </h3>
<h4>Books and Collections</h4>
<ul class="hanging">
<li>Barker, Stephen and Thomas Beauchamp (eds.), 1976, <em>Thomas
Reid: Critical Interpretations</em>, Philadelphia: Philosophical
Monographs.</li>
<li>Berkeley, George, 1948–1957, <em>The Works of George Berkeley,
Bishop of Cloyne</em>. A.A. Luce and T.E. Jessop (eds.). London,
Thomas Nelson and Sons. 9 vols.</li>
<li>Callergård, Robert, 2006, <em>An Essay on Thomas Reid's
Philosophy of Science</em>, Stockholm: Stockholm University
Press.</li>
<li>Cuneo, T. and R. Van Woudenberg, (eds.), 2004, <em>The Cambridge
Companion to Thomas Reid</em>. New York: Cambridge.</li>
<li>Dalgarno, Melvin and Eric Matthews (eds.), 1989, <em>The
Philosophy of Thomas Reid</em>, Dordrecht: Kluwer.</li>
<li>Daniels, Norman, 1974, <em>Thomas Reid's Inquiry: The Geometry of
Visibles and the Case for Realism</em>, New York: Burt Franklin.</li>
<li>Davis, William C., 2006, <em> Thomas Reid's Ethics: Moral
Epi

### Data Extraction

In [4]:
# Start extracting year, author, and title from ref texts

# ======================
# Note that some manual cleaning need to be done here; based on missing years
# We do the cleaning here because some ref can just be deleted 
# =======================

# open data
with open('get_data/ent_ref_dict.json', 'r', encoding='utf-8') as f:
    ent_ref_dict = json.load(f)
    
update_ent_ref_dict = {}


for ent_url in list(ent_ref_dict.keys()):
    # initialize the new dictionary
    update_ent_ref_dict[ent_url] = {}
    # we also want to differentiate each section in the bibliography
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list:
        
        # now, update the dictionary (add details)
        update_ent_ref_dict[ent_url][cat] = {
            "ori_ref_count": ent_ref_dict[ent_url][cat]["ref_count"],
            "ref_count": 0,
            "ref_list": []
        }
        listed_ref = ent_ref_dict[ent_url][cat]["ref_list"]
        for target in listed_ref:

            # escape cases when the ref text is wrong
            # [DCW], a single character, etc.
            if "<ul class=\"hanging\">" in target or "</h3>" in target or len(target) < 8:
                continue
            # E.g., (145) "<font color=\"gray\">Complete translations of seven of Vasubandhu's\nmost
            if re.findall('<font color=\"gray\">', target, re.DOTALL):
                target = target.replace("<font color=\"gray\">", "").replace("</font>", "")
            # E.g., <span class=\"tag\">[TJ]</span> <span class=\"sen\"> <em>A Theory of\nJustice</em>
            if re.findall('<span class=\"tag\">', target, re.DOTALL):
                target = target.replace("<span class=\"tag\">", "").replace("<span class=\"sen\">", "").replace("</span>", "")
            # Note that we cannot remove this because it has <em>, so we just filter it in the title
            # E.g., Explanations on\nthe <em class=\"plain\">Changes</em> of Zhou Dynasty</em>
            if re.findall('<em class=\"plain\">', target, re.DOTALL):
                first_idx = target.index("<em class=\"plain\">")
                last_index = target.index(re.findall('<em class=\"plain\">', target, re.DOTALL)[0]) + len('<em class=\"plain\">')
                # remove the first </em>
                target_after = target[last_index:]
                target = target[:first_idx] + target_after.replace("</em>", "", 1)
            # further clean
            # e.g., <a name=\"Sig93\">Sigmund, K. (1993)</a>.
            if re.findall(r'<a name=\"\S+\">(.*?)</a>', target, re.DOTALL):
                with_style = re.findall(r'<a name=\"\S+\">.*?</a>', target, re.DOTALL)[0]
                no_style = re.findall(r'<a name=\"\S+\">(.*?)</a>', target, re.DOTALL)[0].strip() # get the text within the tag, here it is Sigmund, K. (1993)
                target = target.replace(with_style, no_style)

            # e.g, 
            # if re.findall('[(.*?)]', target, re.DOTALL):
            #     continue

            author, year, title, parent_title = preprocessing.extract_info_from_ref(target)

            update_ent_ref_dict[ent_url][cat]['ref_list'].append(
                {
                    "ref":target,
                    "title": title,
                    "parent_title": parent_title,
                    "author": author,
                    "year": year
                }
            )
            update_ent_ref_dict[ent_url][cat]["ref_count"] = len(update_ent_ref_dict[ent_url][cat]['ref_list'])
            

# print(json.dumps(update_ref_list, indent = 4))

# save
with open('get_data/ent_ref_auth_year_titl_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)
with open('get_data/to_check/check_year_list.json', 'w') as f:
    json.dump(year_check_list, f, indent=4)


ERORR NO [ARCTILE/BOOK] TITLE:  [DMP]/[DSC]

ERORR NO [ARCTILE/BOOK] TITLE:  Thomas S. Maloney, Compendium of the Study of Philosophy, Oxford
University Press for The British Academy, 2018 [Auctores Britannici
Medii Aevi, 32]</tr>
<tr>

ERORR NO [ARCTILE/BOOK] TITLE:  Nikolaus Egel, Edition and Translation, Roger Bacon: Opus
Tertium, Hamburg: Felilx Meiner Verlag, 2019.</tr>
<tr>

ERORR NO [ARCTILE/BOOK] TITLE:  Nilolaus Egel and Katherina Molner, Roger Bacon: Opus maius,
Hamburg: Felix Meiner Verlag, 2017.</tr>
<tr>

ERORR NO [ARCTILE/BOOK] TITLE:  Nikolaus Egel, Roger Bacon: Kompendium fur das Studium der
Philosophie, Hamburg: Felix Meiner Verlag, 2015.</tr>
<tr>

ERORR NO [ARCTILE/BOOK] TITLE:  Pia A. Antilic-Piper, Opus Maius (Selected Texts), Latin-German,
Freiburg-Basel-Wein: Herder Verlag, 2008.</tr>
<tr>

ERORR NO [ARCTILE/BOOK] TITLE:  V.Sorge and F. Seller, Filosofia, scienza, theologia dall’
Opus Maius, Rome: Armando Editore, 2010.</tr>
<tr>

ERORR NO [ARCTILE/BOOK] TITLE: 

### Separate Authors

In [5]:
# Separate authors + add title+parent_title

# open
with open('get_data/ent_ref_auth_year_titl_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)

update_ent_ref_dict = {}

for ent_url in list(ent_ref_dict.keys()):
    # initialize the new dictionary
    update_ent_ref_dict[ent_url] = {}
    # we also want to differentiate each section in the bibliography
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list:
        # now, update the dictionary (add details)
        update_ent_ref_dict[ent_url][cat] = {
            "ori_ref_count": ent_ref_dict[ent_url][cat]["ori_ref_count"],
            "ref_count": ent_ref_dict[ent_url][cat]["ref_count"],
            "ref_dict": {}
        }

        listed_ref = ent_ref_dict[ent_url][cat]["ref_list"]

        for item in listed_ref:

            ref = item["ref"]
            year = item["year"]
            author = item["author"]
            title = item["title"]
            parent_title = item["parent_title"]
            sep_list = []


            if "(ed" in author: # authors are always before (eds.) or (ed.) 
                author = author.split("(ed")[0].strip()
            if " Ed." in author or " ed." in author: # ", Ed." ", ed."
                author = author.split(" Ed.")[0].strip()
                author = author.split(" ed.")[0].strip()
            # have no idea why there are titles but just clean it again
            if "“" in author: 
                if re.findall('\u201c(.*?)\u201d', author, re.DOTALL):
                    title = re.findall('\u201c(.*?)\u201d', author, re.DOTALL)[0].strip()
                else:
                    title = author.split("\u201c")[1].strip()
                author = author.split("“")[0].strip()
            if "<em>" in author:
                if re.findall('<em>(.*?)</em>', author, re.DOTALL):
                    title = re.findall('<em>(.*?)</em>', author, re.DOTALL)[0].strip()
                else:
                    title = author.split("<em>")[1].strip()
                author = author.split("<em>")[0].strip()
            if "<li>" in author:
                if re.findall('<li>(.*?)</li>', author, re.DOTALL):
                    title = re.findall('<li>(.*?)</li>', author, re.DOTALL)[0].strip()
                else:
                    title = author.split("<li>")[1].strip()
                author = author.split("<li>")[0].strip()
            if "<i>" in author:
                if re.findall('<i>(.*?)</i>', author, re.DOTALL):
                    title = re.findall('<i>(.*?)</i>', author, re.DOTALL)[0].strip()
                else:
                    title = author.split("<i>")[1].strip()
                author = author.split("<i>")[0].strip()
            if "‘" in author:
                author = author.split("‘")[0].strip()
                # title = re.findall(r'\s*(\d{4})[a-z]*', author)[0]
            # "Wittgenstein, Ludwig [d. 1951],
            if "[" in author:
                author = author.split("[")[0].strip()
            

            # separate authors 
            sep_list = preprocessing.sep_author(author)
            # print("sep_list:", sep_list)
                
            unique_id = str(uuid.uuid4())
            update_ent_ref_dict[ent_url][cat]['ref_dict'][unique_id] = {
                    "ref":ref,
                    "title": title,
                    "parent_title": parent_title,
                    "title_in_title": title+" | "+parent_title,
                    "year": year,
                    # "author": author,
                    "author_list": sep_list
                }
print("update_ref_list:", len(update_ent_ref_dict))

# # save
with open('get_data/ent_ref_sep_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)


update_ref_list: 1796


## Check missing author names

In [6]:
import json
import uuid
import re
import pickle
import pandas as pd
import copy

import plotly.express as px


import preprocessing
from preprocessing import *

with open('get_data/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)
# print ref that do not have authors
update_ent_ref_dict = copy.deepcopy(ent_ref_dict)
for ent_url in list(update_ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(update_ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = update_ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            ref_dict = all_ref_dict[ref_id]
            author_list = ref_dict["author_list"]
            if not author_list:
                # print(ref_dict["ref"])
                target = ref_dict["ref"]
                if re.findall('<.*\">', target, re.DOTALL):
                    target = re.sub('(<.*\">)', '', target, re.DOTALL)
                # available online</a>] or [<em>Inquiry</em>] or [<b>HKA</b>] 
                if re.findall('\[.*>\]', target, re.DOTALL):
                    target = re.sub('(\[.*\][^\s\w\d]*)', '', target, re.DOTALL)

                author, year, title, parent_title = preprocessing.extract_info_from_ref(target)
                # print(sep_author(author))
                # update
                update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["year"] = year
                update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["title"] = title
                update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["parent_title"] = parent_title
                update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["author"] = author

                update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["author_list"] = preprocessing.sep_author(author)
                update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["ref"] = target


with open('get_data/ent_ref_sep_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)
                


ERORR NO [ARCTILE/BOOK] TITLE:  [DMP]/[DSC]

ERORR NO [ARCTILE/BOOK] TITLE:  (1966a) “Lettre sur la connaissance de l’art
(réponse à André Daspre) (In Ecrits
philosophiques et politiques, Tome II, edited by François
Matheron, 559–68. Ecrits philosophiques et politiques. Paris:
Stock IMEC, 1995.

ERORR NO [ARCTILE/BOOK] TITLE:  (1976b) Etre marxiste en philsophie (Paris: Presses universitaires
de France 2016) tr. as How to Be a Marxist in Philosophy, by G.M.
Goshgarian (London: Bloomsbury, 2017).

ERORR NO [ARCTILE/BOOK] TITLE:  <strong>L</strong>,<strong>C</strong>

ERORR NO [ARCTILE/BOOK] TITLE:  ca.510–15

ERORR NO [ARCTILE/BOOK] TITLE:  SPP1: volume 1

ERORR NO [ARCTILE/BOOK] TITLE:  SPP2: volume 2
</ul> 
</ul>


ERORR NO [ARCTILE/BOOK] TITLE:  [HU] Facsimile of Mersenne’s own annotated copy from the
Bibliothèque des Arts et métiers, introduction by
François Lesure. Paris, 1963 (3 volumes).
</ul>

ERORR NO [ARCTILE/BOOK] TITLE:  <strong>T</strong>

ERORR NO [ARCTILE/BOOK] TITLE:  <

In [7]:
for ent_url in list(update_ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(update_ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = update_ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            ref_dict = all_ref_dict[ref_id]
            author_list = ref_dict["author_list"]
            ref = ref_dict["ref"]
            if not author_list and ref.strip().startswith("<em>"):
                print(ent_url, " | ", cat, "======\n", ref_dict, "\n")
                break

karl-reinhold  |  Selected Secondary Literature About Reinhold ======
 {'ref': '<em>Versuch einer Auflösung der von der philosophischen\nClasse der königl. Akademie der Wissenschaften in Berlin für\n1805 aufgestellten Aufgabe: “Die Natur der Analysis und der\nanalytischen Methode in der Philosophie genau anzugeben, und zu\nuntersuchen: ob und was es für Mittel gebe, ihren Gebrauch\nsicher, leichter und nützlicher zu machen</em>”\n(1805).', 'title': 'Die Natur der Analysis und der\nanalytischen Methode in der Philosophie genau anzugeben, und zu\nuntersuchen: ob und was es für Mittel gebe, ihren Gebrauch\nsicher, leichter und nützlicher zu machen</em>', 'parent_title': '', 'title_in_title': 'Die Natur der Analysis und der\nanalytischen Methode in der Philosophie genau anzugeben, und zu\nuntersuchen: ob und was es für Mittel gebe, ihren Gebrauch\nsicher, leichter und nützlicher zu machen</em> | ', 'year': '1805', 'author_list': [], 'author': ''} 

telesio  |  Secondary literature ======
 

In [ ]:

# if tehre is no author, and
# contain the last name
# or "primary"



## Author Disambiguition


## Update ref

In [8]:
# further clean the ref
with open('get_data/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)

update_ent_ref_dict = {}

# we don't use uuid because I am not sure if there are correct
for ent_url in list(ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            ref_dict = all_ref_dict[ref_id] # the ref, which is a dict containing all the information of the reference
            ref = ref_dict["ref"]
            # "ref": "<li id=\"b:Bicchieribook\">Bicchieri, 
            if re.findall('<.*>', ref, re.DOTALL):
                ref = re.sub('<.*>', '', ref, re.DOTALL)
            ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["ref"] = ref
            
update_ent_ref_dict = copy.deepcopy(ent_ref_dict)
with open('database/sep_clean/ent_ref_sep_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)

## Update title

In [9]:
# remove parent title from title

with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)
update_ent_ref_dict = {}

# we don't use uuid because I am not sure if there are correct
for ent_url in list(ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            ref_dict = all_ref_dict[ref_id] # the ref, which is a dict containing all the information of the reference
            title = ref_dict["title"]
            parent_title = ref_dict["parent_title"]
            title_in_title = ref_dict["title_in_title"]
            
            if re.findall('<em>(.*?)</em>', title, re.DOTALL):
                parent_title = re.findall('<em>(.*?)</em>', title, re.DOTALL)[0].strip()
                title = re.sub('(<em>.*?</em>)', '', title, re.DOTALL).strip()
            elif re.findall('<i>(.*?)</i>', title, re.DOTALL):
                parent_title = re.findall('<i>(.*?)</i>', title, re.DOTALL)[0].strip()
                title = re.sub('(<i>.*?</i>)', '', title, re.DOTALL).strip()
            if title and title[-1] == ',':
                title = title[:-1]
            if parent_title != "":
                title_in_title = title + ' |in| ' + parent_title
            else:
                title_in_title = title

            ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["title"] = title
            ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["parent_title"] = parent_title
            ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["title_in_title"] = title_in_title
            
update_ent_ref_dict = copy.deepcopy(ent_ref_dict)
with open('database/sep_clean/ent_ref_sep_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)

In [ ]:

# resolve 
# "Sleeping Beauty: Reply to
# Elga |in| Analysis (2001) ['David K. Lewis']",
#       "Sleeping Beauty: reply to
# Elga |in| Analysis (2001) ['David K. Lewis']",







In [10]:
# =============================
# Note: NEED TO run the code to get the author names and TITLES!!!!
# =============================

from exploratory_analysis import ref_var_count_with_id, vis_single_var
# %%%%%%%%%%%% run this because we need the title dict to do the disambiguation
with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)
    
var = "title_in_title"
is_list = False
exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
# exploratory_analysis.vis_single_var(var, 100)

var = "author_list"
is_list = True
exploratory_analysis.ref_var_count_with_id(ent_ref_dict, var, is_list) # this will create a dict for vis_single_var to open
# exploratory_analysis.vis_single_var(var, 100)


Length of ent ref dict: 1796
The input variable is title_in_title. Is it a list? False
title_in_title list: 200328
unique title_in_title list: 165913
Length of ent ref dict: 1796
The input variable is author_list. Is it a list? True
author_list list: 241113
unique author_list list: 79061


In [11]:
# disambiguition

# open
with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)
# create a dictionary of title_ref_dict by adding more information to the title_in_title_dict
with open('database/single_variable_list/title_in_title_dict.json', 'r') as f:
    title_dict = json.load(f) # note that this is title_in_title

title_ref_dict = {}
# open the title dict
with open('database/single_variable_list/uni_title_in_title_id_dict.json', 'r') as f:
    uni_title_id_dict = json.load(f)
with open('database/single_variable_list/uni_title_in_title_list.pickle', 'rb') as f:
    uni_title_list = pickle.load(f)


# it the title is cited more than once

for title in uni_title_list:
    title_id = uni_title_id_dict[title]
    if title_dict[title_id]["ref_count"] > 1:
        title_ref_dict[title] = {
            "name": title_dict[title_id]["name"],
            "ref_dict_count": 0,
            "ref_dict": {}
        }
rep_title_list = list(title_ref_dict.keys())
# add the ref_dict to the title_ref_dict
for ent_url in ent_ref_dict:
    for cat in ent_ref_dict[ent_url]:
        for ref_id in ent_ref_dict[ent_url][cat]["ref_dict"]:
            ref = ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]
            title = ref["title"]
            if title in rep_title_list:
                title_ref_dict[title]["ref_dict"][ref_id] = ref
                title_ref_dict[title]["ref_dict_count"] += 1

# print(json.dumps(title_ref_dict, indent = 4))

# save
with open('database/two_variable_dict/title_in_title_ref_dict.json', 'w') as f:
    json.dump(title_ref_dict, f, indent=4)

In [12]:
# find the same authors?

with open('database/two_variable_dict/title_in_title_ref_dict.json', 'r') as f:
    title_ref_dict = json.load(f)
# # open stanford_encyclopedia\database\single_variable_list\ref_dict.json
# with open('database/single_variable_list/ref_dict.json', 'r') as f:
#     ref_dict = json.load(f)
# ERROR: UUID does not match

author_ambi_list_1 = {} # same_title_year_ln_diff_fn_list
author_ambi_list_2 = [] # same_title_year diff ln

# pop the wrok with same titles and same year 
for title in list(title_ref_dict.keys()):
    if title == "":
        continue
    all_ref_dict = title_ref_dict[title]["ref_dict"]

    author_ambi_list_1[title] = {}

    author_year_list = [] # all ref with same title and year, for a specific title
    for ref_id in all_ref_dict:
        ref = all_ref_dict[ref_id] # the ref, which is a dict containing all the information of the reference
        year = ref["year"]
        author_list = ref["author_list"]
        author_year_list.append([year, author_list, ref])

        author_ambi_list_1[title][year] = []
    
    # find items with same year
    for i in range(len(author_year_list)):
        author_year_ref = author_year_list[i]
        year = author_year_ref[0]
        author_list = author_year_ref[1]
        ref = author_year_ref[2]

        for j in range(i+1, len(author_year_list)):
            author_year_ref2 = author_year_list[j]
            year2 = author_year_ref2[0]
            author_list2 = author_year_ref2[1]
            ref2 = author_year_ref2[2]

            if year == year2:
                # different author names for the same title and the same year
                if author_list == author_list2:
                    continue
                # check the single authors first
                if author_list and author_list2 and len(author_list) != len(author_list2):
                    # 1. can use this to recover some missing authors
                    # e.g.,'1918. <em>Psychological Principles</em> => 'Ward, J., 1918, <em>Psychological Principles</em>
                    print("\nERROR:")
                    print(ref, "\n", ref2)
                if len(author_list) == 1 and len(author_list2) == 1:
                    author = author_list[0]
                    author_ln = author.split(" ")[-1]
                    author2 = author_list2[0]
                    author2_ln = author2.split(" ")[-1]
                    if author_ln == author2_ln and author[0] == author2[0]: # same last name, same first name characters
                        author_ambi_list_1[title][year].append([[i, author], [j, author2]])
                        # author_ambi_list_1.append(["1", author, ref["ref"], ref["year"]])
                        # author_ambi_list_1.append(["2", author2, ref2["ref"], ref2["year"]])
                    elif author_ln != author2_ln: # different last name; 
                        if ref["title"] == "": # wrong title extracted?
                            continue
                        author_ambi_list_2.append(["1", author, ref["ref"], ref["title"], ref["year"]])
                        author_ambi_list_2.append(["2", author2, ref2["ref"], ref2["title"], ref2["year"]])
                # check the multiple authors
                # elif len(author_list) > 1 and len(author_list2) > 1:
# save
with open('get_data/to_check/same_title_year_ln_diff_fn_list.json', 'w') as f:
    json.dump(author_ambi_list_1, f, indent=4)
with open('get_data/to_check/same_title_year_diff_ln_list.json', 'w') as f:
    json.dump(author_ambi_list_2, f, indent=4)

# filter authors whoes name contains another


ERROR:
{'ref': 'Cobb, John B. Jr. and David Ray Griffin, 1976, Process\nTheology: An Introductory Exposition, Philadelphia: Westminster\nPress.', 'title': 'Process\nTheology: An Introductory Exposition', 'parent_title': '', 'title_in_title': 'Process\nTheology: An Introductory Exposition', 'year': '1976', 'author_list': ['John B. Jr Cobb', 'David Ray Griffin']} 
 {'ref': 'Cobb, John B., Jr. and David Ray Griffin, 1976, Process\nTheology: An Introductory Exposition, Philadelphia, PA: The\nWestminster Press.', 'title': 'Process\nTheology: An Introductory Exposition', 'parent_title': '', 'title_in_title': 'Process\nTheology: An Introductory Exposition', 'year': '1976', 'author_list': ['John B. Cobb', 'Jr', 'David Ray Griffin']}

ERROR:
{'ref': 'Lemmon, E. J. and Dana Scott, 1977, The ‘Lemmon\nNotes’: An Introduction to Modal Logic, Oxford: Basil\nBlackwell.', 'title': 'Lemmon\nNotes', 'parent_title': '', 'title_in_title': 'Lemmon\nNotes', 'year': '1977', 'author_list': ['E. J. Lemmon', '

In [13]:
sim_names = json.load(open('get_data/to_check/same_title_year_ln_diff_fn_list.json', encoding='utf-8'))

pairs = set()
for _,v in sim_names.items():
    for year, j in v.items():
        same_names = set()
        for k in j:
            for i in k:
                same_names.add(i[1])
        if len(same_names) > 0:
            pairs.add(tuple(same_names))

print(len(pairs))
# save pairs as 'get_data/to_check/name_pairs.json'
with open('get_data/to_check/name_pairs.json', 'w') as f:
    json.dump(list(pairs), f)

2751


In [14]:
def find_name_var(name1, name2):
    # only accept "more information of the middle name"
    name1_list = [i.strip() for i in name1.split(' ') if i.strip()!= '']
    name2_list = [i.strip() for i in name2.split(' ') if i.strip()!= '']

    if name1 == name2:
        return True

    if len(name1_list) <= 2 or len(name2_list) <= 2:
        return False
    # same first name and last name
    if name1_list[0] != name2_list[0] or name1_list[-1] != name2_list[-1]:
        return False

    inital1_list = [i[0] for i in name1_list[1:-1]]
    inital2_list = [i[0] for i in name2_list[1:-1]]
    # make sure the order is correct
    # F. H. Crick => F. H. C. Crick, but what if "F. C. H. Crick"
    # if len(inital1_list) >= len(inital2_list):
    #     return "|"+"".join(inital2_list) in "|"+"".join(inital1_list)
    # else:
    #     return "|"+"".join(inital1_list) in "|"+"".join(inital2_list)
    # ['Robert Merrihew Adams', 'Robert M. Adams'] ['Robert Martin Adams', 'Robert M. Adams']
    
    # make sure the lengths are the same
    if "".join(inital1_list) == "".join(inital2_list):
        find_same = True
        for i in range(len(name1_list)):
            # 'Robert Martin Adams' 'Robert Merrihew Adams'
            if "." not in name1_list[i] and "." not in name2_list[i] and name1_list[i] != name2_list[i]:
                find_same = False
                break
        return find_same
    else:
        return False

find_name_var('Robert Merrihew Adams', 'Robert Merrihew Adams')

True

In [17]:

# ===============================
# Not necessary to run this code
# ===============================


# Add more name pairs

# open get_data/to_check/name_pairs.json
with open('get_data/to_check/name_pairs.json', 'r') as f:
    name_pairs_list = json.load(f)

with open('database/single_variable_list/uni_author_list_id_dict.json') as f:
    uni_author_list_id_dict = json.load(f)

unique_authors_set = list(uni_author_list_id_dict.keys())

long_author_list = []
for name in unique_authors_set:
    name_sep = [i.strip() for i in name.split(' ') if i.strip()!= '']
    if len(name_sep) > 2 and len(name_sep) < 8:
        long_author_list.append(name)

name_var_list = []

for i in range(len(long_author_list)):
    for j in range(i+1, len(long_author_list)):
        if find_name_var(long_author_list[i], long_author_list[j]):
            # print(long_author_list[i], long_author_list[j])
            name_var_list.append([long_author_list[i], long_author_list[j]])
# save name_var_list as json
# with open('name_var_list.json', 'w') as f:
#     json.dump(name_var_list, f, indent=4)

# name_var_list = list(set([tuple(i) for i in name_var_list]))

# name_pairs_list += name_var_list
# NOTE that this list has duplicates
with open('get_data/to_check/name_var_list.json', 'w') as f:
    json.dump(name_var_list, f)
print(len(name_var_list))


599


In [18]:


# name_var_list =  ["Willard V. O. Quine", "Willard Van Orman Quine", "W. V. Quine", "W. V. O. Quine", "Willard. Van Quine"]
# # name_var_list = ["F. Recanati", "F. R\u00e9canati", "Fran\u00e7ois R\u00e9canati", "Fran\u00e7ois Recanati", "Francois Recanati"]
# # name_var_list = ["R. Sorabji", "Richard Sorabji", "R. R. K. Sorabji"]
# name_var_list = ["Alan M. Turing", "A.M. Turing", "A. M. Turing", "A. Turing"]


# note: "R. K. Sorabji" should not replace "Richard Sorabji"?
# don't remove ".", in case ["Richard C Lewontin", "Richard C. Lewontin"]
# but, 'Leo. . Strauss' should be replaced by 'Leo Strauss'
# which means (1) "." only appears after a captical letter, (2) " ." does not exist

# shouldn't correct first


with open('get_data/to_check/name_pairs.json', 'r') as f:
    name_pairs_list = json.load(f)

with open('get_data/to_check/name_var_list.json', 'r') as f:
    name_var_list = json.load(f)

new_var_list = []

# we choose to merge the second list into the first, not the reverse
# thus it can be more reliable
# after all, they are already the most cited works

for i in range(len(name_pairs_list)):
    name_pair_1 = name_pairs_list[i]
    for j in range(len(name_var_list)):
        name_pair_2 = name_var_list[j]

        new_list = list(name_pair_1 + name_pair_2)
        find_same = False
        # find a list (just one among multiple)
        # for these two pairs
        for name_1 in name_pair_1:
            for name_2 in name_pair_2:
                if find_name_var(name_1, name_2):
                    # ['Robert Merrihew Adams', 'Robert M. Adams'] ['Robert Martin Adams', 'Robert M. Adams']
                    find_same = True
                    break
        if find_same:
            # can ignore some minor errors; compared with the errors where the first name is completely missing
            # check = True
            # for i in range(len(new_list)):
            #     for j in range(i+1, len(new_list)):
            #         if len(new_list[i].split(" ")) == len(new_list[j].split(" ")):
            #             if not find_name_var(new_list[i], new_list[j]):
            #                 check = False
            #                 break
            # if check:
            #     name_pair_1 = list(set(new_list))
            #     print(name_pair_1)
            # else:
            #     print("error", name_pair_1, name_pair_2, "|due to|", new_list[i], new_list[j])
            name_pairs_list[i] = list(set(new_list))
            # print(name_pairs_list[i])
            # new_var_list.append(new_list)

print(len(name_pairs_list))

# name_pairs_list

name_var_dict = {}

for name_var_list in name_pairs_list: # for each name variant, 
    for name in name_var_list:
        name_var_dict[name] = []

for name_var_list in name_pairs_list:
    # target_name = target_name_var(name_var_list)
    for name in name_var_list:
        # sometimes there are multiple lists
        name_var_dict[name] += name_var_list
        name_var_dict[name] = list(set(name_var_dict[name]))
name_var_dict
# name_var_all_list is flattened name_pairs
name_var_all_list = list(set([item for sublist in name_pairs_list for item in sublist]))

# save the dict
with open('get_data/to_check/name_var_dict.json', 'w') as f:
    json.dump(name_var_dict, f, indent=4)
    


2751


In [19]:
# update the names


with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)
update_ent_ref_dict = copy.deepcopy(ent_ref_dict)

# we don't use uuid because I am not sure if there are correct
for ent_url in list(update_ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(update_ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = update_ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            ref_dict = all_ref_dict[ref_id] # the ref, which is a dict containing all the information of the reference
            
            year = ref_dict["year"]
            author_list = ref_dict["author_list"]
            update_author_list = []
            if author_list:
                for author in author_list:
                    if author in name_var_all_list: # update target names
                        name_var_list = name_var_dict[author]
                        target_name = target_name_var(name_var_list)

                        # To be moved elsewhere later
                        if target_name == "Hillary Putnam":
                            target_name = "Hilary Putnam"

                        update_author_list.append(target_name)
                    else:
                        update_author_list.append(author)
            else:
                update_author_list = author_list
            update_ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["update_author_list"] = update_author_list

# save
with open('database/sep_clean/ent_ref_sep_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)


### Here we need to manually check typos and correct them

## Add Cleaned Ref

Cleaned_ref = author list + year + title

In [20]:
with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)
update_ent_ref_dict = {}


# we don't use uuid because I am not sure if there are correct
for ent_url in list(ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            ref_dict = all_ref_dict[ref_id] # the ref, which is a dict containing all the information of the reference
            
            title_in_title = ref_dict["title_in_title"]
            year = ref_dict["year"]
            author_list = ref_dict["author_list"]
            update_author_list = ref_dict["update_author_list"]

            update_ref = title_in_title + " (" + year + ") " + str(update_author_list)
            title_year = title_in_title + " (" + year + ")"

            ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["update_ref"] = update_ref
            ent_ref_dict[ent_url][cat]["ref_dict"][ref_id]["title_year"] = title_year

update_ent_ref_dict = copy.deepcopy(ent_ref_dict)
with open('database/sep_clean/ent_ref_sep_dict.json', 'w') as f:
    json.dump(update_ent_ref_dict, f, indent=4)

## Convert to csv

In [21]:
with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
    ent_ref_dict = json.load(f)

ent_ref_info_list = []
# we don't use uuid because I am not sure if there are correct
for ent_url in list(ent_ref_dict.keys()): # for each entry
    cat_ref_list = list(ent_ref_dict[ent_url].keys())
    for cat in cat_ref_list: # for each categories in bibliogrpahies
        all_ref_dict = ent_ref_dict[ent_url][cat]["ref_dict"]
        for ref_id in list(all_ref_dict.keys()):
            info_dict = {}

            ref_dict = all_ref_dict[ref_id] # the ref, which is a dict containing all the information of the reference
            
            info_dict = {
                # ['ref', 'title', 'parent_title', 'title_in_title', 'year', 'author_list', 'update_author_list', 'update_ref']
                "ref_text": ref_dict["update_ref"],
                "cited_by_ent": ent_url,
                "bibli_cat": cat,
                "ref_unique_id": ref_id,
                "title": ref_dict["title_in_title"],
                "year": ref_dict["year"],
                "author_list": "; ".join(ref_dict["update_author_list"])
                # "cited_by_ent": ent_url + ": " + cat,
            
            }

            ent_ref_info_list.append(info_dict)


# in ent_ref_info_list, find the duplicated ref_text and merge them
# we don't use uuid because I am not sure if there are correct
# ent_ref_merged_list = []
# ref_text
# for i in range(len(ent_ref_info_list)):
#     info_dict = ent_ref_info_list[i]
#     ref_text = info_dict["ref_text"]

# convert to csv
df = pd.DataFrame(ent_ref_info_list)
# add df header
df = df[['ref_text', 'cited_by_ent', 'bibli_cat', 'ref_unique_id', 'title', 'year', 'author_list']]
df.to_csv('database/sep_clean/ent_ref_info_list.csv', index=False)
df

,ref_text,cited_by_ent,bibli_cat,ref_unique_id,title,year,author_list
0,Models as\nFictions |in| Fictions in Science. ...,simulations-science,ref_list,86fd4e4d-d6e8-4647-9cfe-6f53c72cb561,Models as\nFictions |in| Fictions in Science. ...,2009,A. Barberousse; P. Ludwig
1,About the warrants of\ncomputer-based empirica...,simulations-science,ref_list,3ad5b266-d13e-4158-9672-499d0a775db3,About the warrants of\ncomputer-based empirica...,2014,A. Barberousse; M. Vorms
2,Weak emergence and computer\nsimulation |in| M...,simulations-science,ref_list,67d5b8a1-0dc3-4fc8-9c94-f31674362526,Weak emergence and computer\nsimulation |in| M...,2011,M. A. Bedau
3,Weak Emergence |in| Noûs (1997) ['M. A. Bedau'],simulations-science,ref_list,3faf6b48-fbac-4783-93e6-02e8f4d071a4,Weak Emergence |in| Noûs,1997,M. A. Bedau
4,Why Monte Carlo\nSimulations are Inferences an...,simulations-science,ref_list,4d43715e-8101-4cd6-a704-becad3cb9c75,Why Monte Carlo\nSimulations are Inferences an...,2012,C. Beisbart; J. Norton
...,...,...,...,...,...,...,...
200323,G. T. Fechner: Premises\ntoward a General Theo...,fechner,Secondary Sources,724cf3c9-bd0f-40b3-8293-f874a4f2aec8,G. T. Fechner: Premises\ntoward a General Theo...,1974,Marilyn E. Marshall
200324,"Im Geiste Fechners, Fünf\nnaturwissenschaftlic...",fechner,Secondary Sources,4029511b-9d1b-4347-b1e3-827afd8c4708,"Im Geiste Fechners, Fünf\nnaturwissenschaftlic...",1901,Willy Pastor
200325,Gustav Theodor Fechner und die\ndurch ihn ersc...,fechner,Secondary Sources,2eaa944a-78b8-4c42-abe1-2ff03f5e7be2,Gustav Theodor Fechner und die\ndurch ihn ersc...,1901,Willy Pastor
200326,Fechner und Lotze (1924) ['Max Wentscher'],fechner,Secondary Sources,eeb15038-beaa-47e9-abf7-c7d33e735daf,Fechner und Lotze,1924,Max Wentscher


In [22]:
# load 'database/sep_clean/ent_ref_info_list.csv'
df = pd.read_csv('database/sep_clean/ent_ref_info_list.csv')

# open database\single_variable_list\uni_title_in_title_list.pickle
# with open('database/single_variable_list/uni_update_ref_list.pickle', 'rb') as f:
#     uni_update_ref_list = pickle.load(f)

uni_update_ref_list = list(set(df["ref_text"]))

# print(json.dumps(uni_update_ref_list, indent=4))
# for ref_text in df, build a dictioanry of its information
ref_text_dict = {}

for ref_text in uni_update_ref_list:
    ref_text_dict[ref_text] = {
        "cited_by_ent_list": []
    }

for i in range(len(df)):
    ref_text = df.iloc[i]["ref_text"]
    cited_ent = df.iloc[i]["cited_by_ent"]
    cited_by_ent_list = ref_text_dict[ref_text]["cited_by_ent_list"]
    cited_by_ent_list += [cited_ent]

    ref_text_dict[ref_text] = {
        "title": df.iloc[i]["title"],
        "year": df.iloc[i]["year"],
        "author_list": df.iloc[i]["author_list"],
        "ref_unique_id": df.iloc[i]["ref_unique_id"],
        "cited_by_ent_list": cited_by_ent_list
    }


print(json.dumps(ref_text_dict, indent=4))

In [ ]:
# # from database/sep_clean/ent_ref_info_list.csv
# # find the same reference
# df = pd.read_csv('database/sep_clean/ent_ref_info_list.csv')
# # find references with same ref_text
# for ref_text in list(df['ref_text'].unique()):
#     df_ref = df[df['ref_text'] == ref_text]
#     if len(df_ref) > 1:
#         print(df_ref)

In [3]:
df = pd.read_csv('database/sep_clean/ent_ref_info_list.csv')
df

,ref_text,cited_by_ent,bibli_cat,ref_unique_id,title,year,author_list
0,Models as\nFictions |in| Fictions in Science. ...,simulations-science,ref_list,86fd4e4d-d6e8-4647-9cfe-6f53c72cb561,Models as\nFictions |in| Fictions in Science. ...,2009,A. Barberousse; P. Ludwig
1,About the warrants of\ncomputer-based empirica...,simulations-science,ref_list,3ad5b266-d13e-4158-9672-499d0a775db3,About the warrants of\ncomputer-based empirica...,2014,A. Barberousse; M. Vorms
2,Weak emergence and computer\nsimulation |in| M...,simulations-science,ref_list,67d5b8a1-0dc3-4fc8-9c94-f31674362526,Weak emergence and computer\nsimulation |in| M...,2011,M. A. Bedau
3,Weak Emergence |in| Noûs (1997) ['M. A. Bedau'],simulations-science,ref_list,3faf6b48-fbac-4783-93e6-02e8f4d071a4,Weak Emergence |in| Noûs,1997,M. A. Bedau
4,Why Monte Carlo\nSimulations are Inferences an...,simulations-science,ref_list,4d43715e-8101-4cd6-a704-becad3cb9c75,Why Monte Carlo\nSimulations are Inferences an...,2012,C. Beisbart; J. Norton
...,...,...,...,...,...,...,...
200323,G. T. Fechner: Premises\ntoward a General Theo...,fechner,Secondary Sources,724cf3c9-bd0f-40b3-8293-f874a4f2aec8,G. T. Fechner: Premises\ntoward a General Theo...,1974,Marilyn E. Marshall
200324,"Im Geiste Fechners, Fünf\nnaturwissenschaftlic...",fechner,Secondary Sources,4029511b-9d1b-4347-b1e3-827afd8c4708,"Im Geiste Fechners, Fünf\nnaturwissenschaftlic...",1901,Willy Pastor
200325,Gustav Theodor Fechner und die\ndurch ihn ersc...,fechner,Secondary Sources,2eaa944a-78b8-4c42-abe1-2ff03f5e7be2,Gustav Theodor Fechner und die\ndurch ihn ersc...,1901,Willy Pastor
200326,Fechner und Lotze (1924) ['Max Wentscher'],fechner,Secondary Sources,eeb15038-beaa-47e9-abf7-c7d33e735daf,Fechner und Lotze,1924,Max Wentscher


In [4]:
# from the csv
df = pd.read_csv('database/sep_clean/ent_ref_info_list.csv')

# break down the author list
author_count_dict = {}
author_entry_dict = {}
for i in range(len(df)):
    author = df.iloc[i]['author_list']
    # find the entry url
    ent_url = df.iloc[i]['cited_by_ent'].split(': ')[0]
    if type(author) != float:
        # author_list += author.split('; ')
        # count
        for a in author.split('; '):
            if a in author_count_dict:
                author_count_dict[a] += 1
            else:
                author_count_dict[a] = 1
        for a in author.split('; '):
            if a in author_entry_dict:
                author_entry_dict[a] += [ent_url]
            else: 
                author_entry_dict[a] = [ent_url]
# author_list = list(set(author_list))
# author_list = [author.strip() for author in author_list if author != '']

# the output is a dictionary of cited authors and the number of times she/he gets cited
author_cited_dict = {}
for author in author_count_dict:
    ent_list = list(set(author_entry_dict[author]))
    # set as a dict that counts the number of times each entry cites the author
    ent_count_dict = {}
    for ent in ent_list:
        ent_count_dict[ent] = author_entry_dict[author].count(ent)
    # sort the dict by the number of times each entry cites the author
    ent_count_dict = dict(sorted(ent_count_dict.items(), key=lambda item: item[1], reverse=True))
    # add to the output dict

    author_cited_dict[author] = {
        'name': author,
        'cited_count': author_count_dict[author],
        'cited_ent_list': "; ".join(list(ent_count_dict.keys())),
        'cited_ent_dict': str(ent_count_dict)
    }
# sort and save as json
author_cited_dict = dict(sorted(author_cited_dict.items(), key=lambda item: item[1]['cited_count'], reverse=True))
with open('database/sep_clean/author_cited_count.json', 'w') as f:
    json.dump(author_cited_dict, f, indent=4)

# dict to df
author_cited_df = pd.DataFrame.from_dict(author_cited_dict, orient='index', columns=['name', 'cited_count', 'cited_ent_list', 'cited_ent_dict'])
# sort by cited_count
author_cited_df = author_cited_df.sort_values(by=['cited_count'], ascending=False)
# save to csv
author_cited_df.to_csv('database/sep_clean/author_cited_count.csv', index=False)
author_cited_df


,name,cited_count,cited_ent_list,cited_ent_dict
David K. Lewis,David K. Lewis,879,david-lewis; lewis-metaphysics; natural-proper...,"{'david-lewis': 136, 'lewis-metaphysics': 33, ..."
Bertrand Russell,Bertrand Russell,664,logical-atomism; russell; russell-moral; neutr...,"{'logical-atomism': 111, 'russell': 50, 'russe..."
Willard van Orman Quine,Willard van Orman Quine,485,quine; carnap; analytic-synthetic; epistemolog...,"{'quine': 30, 'carnap': 15, 'analytic-syntheti..."
Immanuel Kant,Immanuel Kant,403,kant-religion; kant-metaphysics; kant-reason; ...,"{'kant-religion': 25, 'kant-metaphysics': 13, ..."
Hilary Putnam,Hilary Putnam,366,realism-sem-challenge; wittgenstein-mathematic...,"{'realism-sem-challenge': 13, 'wittgenstein-ma..."
...,...,...,...,...
Mark Perlman,Mark Perlman,1,teleology-biology,{'teleology-biology': 1}
T. M. Caro,T. M. Caro,1,teleology-biology,{'teleology-biology': 1}
Paul Sheldon Davies,Paul Sheldon Davies,1,teleology-biology,{'teleology-biology': 1}
B. D. Davis,B. D. Davis,1,teleology-biology,{'teleology-biology': 1}


In [ ]:
# open 'database/sep_clean/author_cited_count.csv'

author_cited_df = pd.read_csv('database/sep_clean/author_cited_count.csv')
# build a df of each pair of author and cited_ent
for i in range(len(author_cited_df)):
    author = author_cited_df.iloc[i]['name']
    cited_ent_list = author_cited_df.iloc[i]['cited_ent_list'].split('; ')
    for ent in cited_ent_list:
        author_cited_df = author_cited_df.append({'name': author, 'cited_ent': ent, }, ignore_index=True)


In [ ]:


with open('database/sep_clean/ent_ref_sep_dict.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('database/sep_clean/ent_ref.csv', encoding='utf-8', index=False)

# check 
pd.read_csv('database/sep_clean/ent_ref.csv')

